# import

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import math
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from  PIL import Image
import os
import random
from tqdm import tqdm
import timm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# 定义参数

In [3]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# stage1

In [2]:
INPUT_PATH = '../input/ensemble-1026/'
TRAIN_CSV_PATH = '../input/cassava-leaf-disease-classification/train.csv'
TEST_IMAGE_PATH = '../input/cassava-leaf-disease-classification/test_images/'
SUBMISSION_PATH = 'submission.csv'
RESNEXT_PATH = '1026_res50d.pth'
B4_PATH = '1026_b5ns.pth'
DEVICES = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
OUT_FEATURES = 5
BATCH_SIZE = 32
SEED = 43
TTA = 8

In [4]:
model_name1 = "resnext50d_32x4d"
my_model_1 = timm.create_model(model_name1, pretrained=False)
my_model_1.fc = nn.Linear(my_model_1.fc.in_features, OUT_FEATURES)
nn.init.xavier_uniform_(my_model_1.fc.weight)
if my_model_1.fc.bias is not None:
    nn.init.zeros_(my_model_1.fc.bias)
my_model_1

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): 

In [5]:
model_name2 = "tf_efficientnet_b5_ns"
my_model_2 = timm.create_model(model_name2, pretrained=False)
my_model_2.classifier = nn.Linear(my_model_2.classifier.in_features, OUT_FEATURES)
nn.init.xavier_uniform_(my_model_2.classifier.weight)
if my_model_2.classifier.bias is not None:
    nn.init.zeros_(my_model_2.classifier.bias)
my_model_2

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name tf_efficientnet_b5_ns to current tf_efficientnet_b5.ns_jft_in1k.
  model = create_fn(


EfficientNet(
  (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
        (bn1): BatchNormAct2d(
          48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
          24, ep

In [7]:
IMAGE_SIZE1 = 512
IMAGE_SIZE2 = 384
seed_everything(SEED)
torch.cuda.empty_cache()
preds_1 = []
test_augs1 = A.Compose([
    A.OneOf([
        A.Resize(IMAGE_SIZE1, IMAGE_SIZE1, p=1.0),
        A.CenterCrop(IMAGE_SIZE1, IMAGE_SIZE1, p=1.0),
        A.RandomResizedCrop(IMAGE_SIZE1, IMAGE_SIZE1, p=1.0)
    ], p=1.0),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(IMAGE_SIZE1, IMAGE_SIZE1),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
    ], p=1.0
)
my_model_1.eval()
model_param = torch.load(os.path.join(INPUT_PATH, RESNEXT_PATH))
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_1.load_state_dict(new_model_param)
my_model_1 = nn.DataParallel(my_model_1).to(DEVICES[0])
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(1):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs1(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_1(test_image).view(ans.shape)
        preds_1.append(ans)
predictions_1 = torch.stack(preds_1, dim=0).to('cpu')
normalize_pred_1 = F.normalize(predictions_1.T, p=2, dim=0).T


torch.cuda.empty_cache()
preds_2 = []
test_augs2 = A.Compose([
    A.OneOf([
        A.Resize(IMAGE_SIZE2, IMAGE_SIZE2, p=1.0),
        A.CenterCrop(IMAGE_SIZE2, IMAGE_SIZE2, p=1.0),
        A.RandomResizedCrop(IMAGE_SIZE2, IMAGE_SIZE2, p=1.0)
    ], p=1.0),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(IMAGE_SIZE2, IMAGE_SIZE2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
    ], p=1.0
)
my_model_2.eval()
model_param = torch.load(os.path.join(INPUT_PATH, B4_PATH))
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_2.load_state_dict(new_model_param)
my_model_2 = nn.DataParallel(my_model_2).to(DEVICES[0])
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(TTA):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs2(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_2(test_image).view(ans.shape)
        ans /= TTA
        preds_2.append(ans)
predictions_2 = torch.stack(preds_2, dim=0).to('cpu')
normalize_pred_2 = F.normalize(predictions_2.T, p=2, dim=0).T
normalize_pred_2 = normalize_pred_2.cpu()

final_pred1 = (normalize_pred_1 * 0.27) + (normalize_pred_2 * 0.73)

/tmp/ipykernel_26/3072095907.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
/tmp/ipykernel_26/3072095907.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()


# stage2

In [ ]:
INPUT_PATH = '../input/ensemble-1023/'
TRAIN_CSV_PATH = '../input/cassava-leaf-disease-classification/train.csv'
TEST_IMAGE_PATH = '../input/cassava-leaf-disease-classification/test_images/'
SUBMISSION_PATH = 'submission.csv'
RESNEXT_PATH = '1022_res50.pth'
B4_PATH = '1022_b4ns.pth'
DEVICES = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
OUT_FEATURES = 5
IMAGE_SIZE = 512
OPTIMIZER = torch.optim.AdamW
SEED = 42
TTA = 8

In [ ]:
test_augs = A.Compose([
    A.OneOf([
        A.Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        A.CenterCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
        A.RandomResizedCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0)
    ], p=1.0),
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Resize(IMAGE_SIZE, IMAGE_SIZE),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
    ToTensorV2(p=1.0)
    ], p=1.0
)

In [ ]:
model_name1 = "resnext50_32x4d"
my_model_1 = timm.create_model(model_name1, pretrained=False)
my_model_1.fc = nn.Linear(my_model_1.fc.in_features, OUT_FEATURES)
nn.init.xavier_uniform_(my_model_1.fc.weight)
if my_model_1.fc.bias is not None:
    nn.init.zeros_(my_model_1.fc.bias)
my_model_1

In [ ]:
model_name2 = "tf_efficientnet_b4_ns"
my_model_2 = timm.create_model(model_name2, pretrained=False)
my_model_2.classifier = nn.Linear(my_model_2.classifier.in_features, OUT_FEATURES)
nn.init.xavier_uniform_(my_model_2.classifier.weight)
if my_model_2.classifier.bias is not None:
    nn.init.zeros_(my_model_2.classifier.bias)
my_model_2

In [ ]:
seed_everything(SEED)
torch.cuda.empty_cache()
preds_1 = []
my_model_1.eval()
model_param = torch.load(os.path.join(INPUT_PATH, RESNEXT_PATH))
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_1.load_state_dict(new_model_param)
my_model_1 = nn.DataParallel(my_model_1).to(DEVICES[0])
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(1):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_1(test_image).view(ans.shape)
        preds_1.append(ans)
predictions_1 = torch.stack(preds_1, dim=0).to('cpu')
normalize_pred_1 = F.normalize(predictions_1.T, p=2, dim=0).T


torch.cuda.empty_cache()
preds_2 = []
my_model_2.eval()
model_param = torch.load(os.path.join(INPUT_PATH, B4_PATH))
new_model_param = {k[7:]: v for k, v in model_param.items() if 'module.' in k}
my_model_2.load_state_dict(new_model_param)
my_model_2 = nn.DataParallel(my_model_2).to(DEVICES[0])
test_image_list = np.asarray([image_name for image_name in os.listdir(TEST_IMAGE_PATH)])
for single_image_name in test_image_list:
    with torch.no_grad():
        ans = torch.zeros(5).cuda()
        for _ in range(TTA):
            image = Image.open(os.path.join(TEST_IMAGE_PATH, single_image_name))
            aug_image = test_augs(image=np.array(image))['image']
            test_image = torch.tensor(aug_image, dtype=torch.float).unsqueeze(0).cuda()
            ans += my_model_2(test_image).view(ans.shape)
        ans /= TTA
        preds_2.append(ans)
predictions_2 = torch.stack(preds_2, dim=0).to('cpu')
normalize_pred_2 = F.normalize(predictions_2.T, p=2, dim=0).T
normalize_pred_2 = normalize_pred_2.cpu()

final_pred2 = (final_pred1 * 0.4) + (((normalize_pred_1 * 0.45) + (normalize_pred_2 * 0.55)) * 0.6)

label = final_pred2.argmax(dim=-1).numpy()
label_list = list(label)
df_submission = pd.DataFrame(columns=pd.read_csv(TRAIN_CSV_PATH).columns)
df_submission['image_id'] = pd.DataFrame(test_image_list)
df_submission['label'] = pd.DataFrame(label_list)
df_submission.to_csv(SUBMISSION_PATH, index=False)